# Tesla specific time series processing
The goal of this notebook is to demonstrate the implementation of time series processing steps that are specific to Tesla.

## Setup

### Imports

In [ ]:
import plotly.express as px

from pandas.api.types import CategoricalDtype

from core.pandas_utils import *
from transform.processed_tss.config import IN_CHARGE_CHARGING_STATUS_VALS, IN_DISCHARGE_CHARGING_STATUS_VALS
from transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries
from transform.raw_results.tesla_results import get_results

### Data extraction

In [ ]:
! mkdir -p data_cache

In [ ]:
COLUMNS = [
    "soc",
    "charging_status",
    "vin",
    "charge_limit_soc",
    "charge_energy_added",
    "sec_time_diff",
    "date",
    "time_diff",
    "odometer",
]
tss = TeslaProcessedTimeSeries(use_cols=COLUMNS) # To get all time series 
# tss = pd.read_parquet("data_cache/tesla_sub_tss.parquet", columns=COLUMNS) # To get a subset... and prevent my laptop from crashing

In [ ]:
# tss.dtypes
# tss = tss.astype({
#     "vin": CategoricalDtype(),
#     "charging_status": CategoricalDtype(),
# })

## Segmentation and indexing

In [ ]:
tss["charging_status"].value_counts(normalize=True, dropna=False)

In [ ]:
MIN_POWER_LOSS = -0.0005
MAX_CHARGE_TD = TD(days=1)

def compute_charge_n_discharge_masks(tss:DF) -> DF:
    tss["nan_charging"] = (
        Series(pd.NA, index=tss.index, dtype="boolean")
        .mask(tss["charging_status"].isin(IN_CHARGE_CHARGING_STATUS_VALS), True)
        .mask(tss["charging_status"].isin(IN_DISCHARGE_CHARGING_STATUS_VALS), False)
    )
    tss["nan_date"] = tss["date"].mask(tss["nan_charging"].isna())
    tss[["ffill_charging", "ffill_date"]] = tss.groupby("vin", observed=True)[["nan_charging", "nan_date"]].ffill()
    tss[["bfill_charging", "bfill_date"]] = tss.groupby("vin", observed=True)[["nan_charging", "nan_date"]].bfill()
    nan_period_duration:Series = tss.eval("bfill_date - ffill_date")
    fill_unknown_period = tss.eval("ffill_charging.eq(bfill_charging) & @nan_period_duration.le(@MAX_CHARGE_TD)")
    tss["nan_charging"] = tss["nan_charging"].mask(fill_unknown_period, tss["ffill_charging"])
    tss["nan_charging"] = tss["nan_charging"].mask(tss["soc"] >= (tss["charge_limit_soc"] - 3))
    tss["in_charge"] = tss.eval("nan_charging.notna() & nan_charging")
    tss["in_discharge"] = tss.eval("nan_charging.notna() & ~nan_charging")
    return tss

def compute_charge_idx(tss:DF) -> DF:
    tss_grp = tss.groupby("vin", observed=False)
    tss["charge_energy_added"] = tss_grp["charge_energy_added"].ffill()
    tss["power_loss"] = tss_grp['charge_energy_added'].diff().div(tss["sec_time_diff"].values)
    new_charge_mask = tss["power_loss"].lt(MIN_POWER_LOSS, fill_value=0) 
    new_charge_mask |= tss["time_diff"].gt(MAX_CHARGE_TD) 
    new_charge_mask |= (~tss_grp["in_charge"].shift() & tss["in_charge"])
    tss["in_charge_idx"] = new_charge_mask.groupby(tss["vin"], observed=True).cumsum()
    return tss

def compute_status_col(tss:DF) -> DF:
    tss_grp = tss.groupby("vin", observed=True)
    tss["status"] = Series(pd.NA, index=tss.index, dtype=CategoricalDtype(["in_charge", "moving", "unknown", "idle_discharging"]))
    tss["status"] = tss["status"].mask(tss["in_charge"], "in_charge")
    tss["status"] = tss["status"].mask(
        tss["in_discharge"], 
        np.where(tss_grp["odometer"].diff() > 0, "moving", "idle_discharging")
    )
    return tss

def trim_leading_n_trailing_soc_off_masks(tss:DF, masks:list[str]) -> DF:
    for mask in masks:
        tss["naned_soc"] = tss["soc"].where(tss[mask])
        soc_grp = tss.groupby(["vin", mask + "_idx"], observed=True)["naned_soc"]
        trailing_soc = soc_grp.transform("first")
        leading_soc = soc_grp.transform("last")
        tss["trailing_soc"] = trailing_soc
        tss["leading_soc"] = leading_soc
        tss[f"trimmed_{mask}"] = tss[mask] & (tss["soc"] != trailing_soc) & (tss["soc"] != leading_soc)
    tss = tss.drop(columns="naned_soc")
    return tss

In [ ]:
tss = (
    tss
    .pipe(compute_charge_n_discharge_masks)
    .pipe(compute_charge_idx)
    .pipe(compute_status_col)
    .pipe(trim_leading_n_trailing_soc_off_masks, ["in_charge"])
)

In [ ]:
# px.scatter(
#     tss.query("vin == 'LRW3E7FA5LC098336'").eval("charging_status_str = charging_status.astype('string').fillna('unknown')"),
#     x="date",
#     y="soc",
#     color="charging_status_str",
#     # symbol="",
#     hover_data=["odometer", "charging_status_str"]
# ).update_layout(showlegend=True)

In [ ]:
charge_lengths:DF = (
    tss
    .query("status == 'in_charge'")
    .groupby(["vin", "in_charge_idx"], observed=True)
    .agg(
        start_date=pd.NamedAgg("date", "first"),
        end_date=pd.NamedAgg("date", "last")
    )
    .eval("duration = end_date - start_date")
    .sort_values(by="duration")
)
charge_lengths

In [ ]:
# tss["in_charge_idx_size"] = tss.groupby(["vin", "in_charge_idx"], observed=True).transform("size") == 1
tss["one_point_charge"] = tss.eval("status == 'in_charge' & in_charge_idx_size == 1")

In [ ]:
single_points_charges_counts = (
    tss
    .groupby("vin", observed=True, as_index=False)
    ["one_point_charge"]
    .sum()
    .sort_values(by="one_point_charge")
    .reset_index(drop=True)
)
single_points_charges_counts

In [ ]:
px.scatter(
    (
        tss
        .query("vin == '5YJSA7E26JF250824'")
        .eval("charging_status = charging_status.astype('string').fillna('unknown')")
        .eval("status = status.astype('string').fillna('unknown')")
        .melt(
            [
                "date",
                "status",
                "in_charge_idx","charging_status", "in_charge", "in_discharge", "one_point_charge", "in_charge_idx_size"], 
            [
                "soc",
                # "power_loss",
            ]
        )
    ),
    x="date",
    y="value",
    facet_row="variable",
    color="one_point_charge",
    symbol="status",
    hover_data=["in_charge_idx", "charging_status", "in_charge", "in_discharge"],
    height=650,
).update_yaxes(matches=None)

In [ ]:
px.histogram(
    single_points_charges_counts,
    x="one_point_charge",
    log_y=True,
    hover_data=["vin"],
)

Usually, very long charging periods are caused by some edge case scenario that prevents our pipeline to correctly segment/separate multiple periods.

In [ ]:
charge_lengths[charge_lengths["duration"].between(TD(hours=15), TD(hours=20))]

In [ ]:
# px.scatter(
#     (
#         tss
#         .query("vin == 'LRW3E7ET4RC173510'")
#         .eval("charging_status = charging_status.astype('string').fillna('unknown')")
#         .eval("nan_period_duration = bfill_date - ffill_date")
#         .eval("nan_period_duration_sec = nan_period_duration.dt.total_seconds() / 3600")
#         .eval("fill_unknown_period = nan_period_duration.le(@MAX_CHARGE_TD)")
#         .eval("status = status.astype('string').fillna('unknown')")
#         .melt(
#             [
#                 "date",
#                 "status",
#                 "in_charge_idx",
#                 "charging_status",
#                 "in_charge",
#                 "nan_period_duration",
#                 "nan_period_duration_sec",
#                 "fill_unknown_period",
#             ],
#             [
#                 "soc",
#                 # "charge_limit_soc",
#                 "charge_energy_added",
#                 "power_loss",
#             ]
#         )
#     ),
#     x="date",
#     y="value",
#     facet_row="variable",
#     color="charging_status",
#     symbol="in_charge",
#     hover_data=["in_charge_idx", "charging_status", "nan_period_duration_sec", "fill_unknown_period"],
#     height=650,
# ).update_yaxes(matches=None)

In [ ]:
# px.histogram(
#     charge_lengths.eval("hours_duration = duration.dt.total_seconds().div(3600)"),
#     x="hours_duration",
#     log_y=True,
# )

Same goes for vehicles that have a lot of charges, sometimes this is due to an error on our end.

In [ ]:
charge_counts = tss.query("in_charge").groupby("vin", observed=True)["in_charge_idx"].nunique().sort_values()
charge_counts

Charging up to 60 and oscilating, vin: `5YJ3E7EB1KF334219`

In [ ]:
# px.scatter(
#     (
#         tss
#         .query("vin == '5YJ3E7EB1KF334219'")
#         .eval("charging_status = charging_status.astype('string').fillna('unknown')")
#         .eval("status = status.astype('string').fillna('unknown')")
#         .melt(
#             ["date", "status", "in_charge_idx", "charging_status", "in_charge", "in_discharge"], 
#             [
#                 "soc",
#                 "power_loss",
#             ]
#         )
#     ),
#     x="date",
#     y="value",
#     facet_row="variable",
#     color="status",
#     # symbol="in_charge_idx",
#     hover_data=["in_charge_idx", "charging_status", "in_charge", "in_discharge"],
#     height=650,
# ).update_yaxes(matches=None)